<a href="https://colab.research.google.com/github/ttogle918/ds-section2-sprint1/blob/main/n213-ridge-regression/n213a-ridge-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 1 / NOTE 3*

# 📝 Assignment
---

# Ridge Regression

이번 모듈에서 배운 내용을 조금 다른 데이터를 통해 복습해 봅시다. 데이터에 대한 정보는 다음 링크를 참조해 주세요.

- [Melbourne Housing Market]('https://www.kaggle.com/anthonypino/melbourne-housing-market')

이번에도 Price 변수를 target으로 정하고 모델을 구성해 봅시다.

In [1]:
# !pip install pandas-profiling==2.11.0 --upgrade
# !pip install category_encoders

# # best_score_ 가 동작 안 할 경우 아래의 코드 실행 후 런타임 재시작 진행
# !pip install --upgrade scikit-learn==0.24.1

In [2]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split

df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/melbourne_house_prices/MELBOURNE_HOUSE_PRICES_LESS.csv')
df.drop(columns=['Suburb','Address','SellerG','Date'], inplace=True)
df.dropna(subset=['Price'], inplace=True)
df.drop_duplicates(inplace=True)
target = 'price'
df.shape

(42515, 9)

### 1) 데이터의 특성을 파악하기 위해 pandas_profiling을 사용해 보세요.
- **범주형 특성으로 원핫인코딩을 할 수 있는 특성의 전체 개수를 과제 제출폼에 제출하세요.**

In [3]:
df.head(2)

,Rooms,Type,Price,Method,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,3,h,1490000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,3,h,1220000.0,S,3067,Northern Metropolitan,4019,3.0,Yarra City Council


In [4]:
# ProfileReport(df)

### 2) 범주형 변수들에 대해 one-hot encoding(category_encoders)을 수행하고 데이터를 train/test 셋으로 무작위로 나누어 주세요.(train/test = 80%/20%)
- [category_encoders.one_hot.OneHotEncoder](https://contrib.scikit-learn.org/category_encoders/onehot.html#one-hot), 파라미터는 use_cat_names = True 사용
- 데이터 분리는 [sklearn.model_selection.train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train%20test%20split#sklearn-model-selection-train-test-split) 을 사용해 주세요. 사용해야 하는 파라미터는 test_size, train_size, random_state=2 입니다. 나머지는 default 사용합니다.
- **분리된 학습데이터의 첫번째 샘플의 인덱스 값를 과제 제출폼에 제출하세요.**


In [5]:
df['Type'].value_counts(normalize=True)

h    0.697095
u    0.196636
t    0.106268
Name: Type, dtype: float64

In [6]:
from sklearn.model_selection import train_test_split
data = df[['Rooms','Type', 'Method', 'Postcode', 'Regionname', 'Propertycount',	'Distance',	'CouncilArea']]
target = df['Price']

x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.2, train_size=0.8, random_state=2)

In [7]:
from category_encoders import OneHotEncoder
## 원핫 인코딩
encoder = OneHotEncoder(use_cat_names = True)
x_train = encoder.fit_transform(x_train)  # fit을 통해 encoder에 data의정보가 흘러들어간다.(data의 특성이 반영된다.)
x_test = encoder.transform(x_test) 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
x_train.head()

,Rooms,Type_h,Type_u,Type_t,Method_S,Method_VB,Method_SP,Method_PI,Method_SA,Postcode,Regionname_Eastern Metropolitan,Regionname_South-Eastern Metropolitan,Regionname_Western Metropolitan,Regionname_Northern Metropolitan,Regionname_Southern Metropolitan,Regionname_Eastern Victoria,Regionname_Western Victoria,Regionname_Northern Victoria,Propertycount,Distance,CouncilArea_Manningham City Council,CouncilArea_Kingston City Council,CouncilArea_Hobsons Bay City Council,CouncilArea_Brimbank City Council,CouncilArea_Whittlesea City Council,CouncilArea_Hume City Council,CouncilArea_Stonnington City Council,CouncilArea_Frankston City Council,CouncilArea_Monash City Council,CouncilArea_Darebin City Council,CouncilArea_Boroondara City Council,CouncilArea_Yarra City Council,CouncilArea_Banyule City Council,CouncilArea_Moreland City Council,CouncilArea_Wyndham City Council,CouncilArea_Port Phillip City Council,CouncilArea_Whitehorse City Council,CouncilArea_Glen Eira City Council,CouncilArea_Bayside City Council,CouncilArea_Moonee Valley City Council,CouncilArea_Melbourne City Council,CouncilArea_Maribyrnong City Council,CouncilArea_Knox City Council,CouncilArea_Maroondah City Council,CouncilArea_Yarra Ranges Shire Council,CouncilArea_Melton City Council,CouncilArea_Greater Dandenong City Council,CouncilArea_Casey City Council,CouncilArea_Macedon Ranges Shire Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Cardinia Shire Council,CouncilArea_Moorabool Shire Council,CouncilArea_Mitchell Shire Council,CouncilArea_Murrindindi Shire Council
602,4,1,0,0,1,0,0,0,0,3106,1,0,0,0,0,0,0,0,6202,15.5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40851,5,1,0,0,1,0,0,0,0,3169,0,1,0,0,0,0,0,0,2727,17.5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
57233,4,1,0,0,0,1,0,0,0,3028,0,0,1,0,0,0,0,0,7630,15.5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8816,4,1,0,0,1,0,0,0,0,3033,0,0,1,0,0,0,0,0,5629,11.7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3481,3,1,0,0,1,0,0,0,0,3076,0,0,0,1,0,0,0,0,10926,19.6,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 3) SelectKBest를 사용해서 20개의 특성만 selection 하세요
- score_func 에는 f_regression을 사용합니다
- **가장 스코어(Attributes: scores_ 사용)가 높게 나온 특성의 이름을 과제 제출폼에 제출하세요.**

In [9]:
x_train.columns[12]

'Regionname_Western Metropolitan'

In [10]:
from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func=f_regression, k=20)
X_train_selected = selector.fit_transform(x_train, y_train)
X_test_selected = selector.transform(x_test)

In [11]:
all_names = x_train.columns
selected_mask = selector.get_support()

selected_names = all_names[selected_mask] # 선택된 특성들
unselected_names = all_names[~selected_mask] # 선택되지 않은 특성들

print('Selected names: ', selected_names)
print('Unselected names: ', unselected_names)

Selected names:  Index(['Rooms', 'Type_h', 'Type_u', 'Method_VB', 'Method_SP',
       'Regionname_South-Eastern Metropolitan',
       'Regionname_Western Metropolitan', 'Regionname_Northern Metropolitan',
       'Regionname_Southern Metropolitan', 'Distance',
       'CouncilArea_Manningham City Council',
       'CouncilArea_Brimbank City Council',
       'CouncilArea_Whittlesea City Council', 'CouncilArea_Hume City Council',
       'CouncilArea_Stonnington City Council',
       'CouncilArea_Frankston City Council',
       'CouncilArea_Boroondara City Council',
       'CouncilArea_Wyndham City Council', 'CouncilArea_Bayside City Council',
       'CouncilArea_Melton City Council'],
      dtype='object')
Unselected names:  Index(['Type_t', 'Method_S', 'Method_PI', 'Method_SA', 'Postcode',
       'Regionname_Eastern Metropolitan', 'Regionname_Eastern Victoria',
       'Regionname_Western Victoria', 'Regionname_Northern Victoria',
       'Propertycount', 'CouncilArea_Kingston City Council',

In [12]:
import numpy as np

score = selector.scores_
best_score_index = np.argmax(score) # 가장 높은 점수의 인덱스 번호 추출
all_names[best_score_index]

'Rooms'

### 4) RidgeCV를 사용해서 회귀모델을 만들어 보세요
- attribute는 normalize=True, cv=5를 사용하세요. 나머지는 default
- **alphas = [0, 0.001, 0.01, 0.1, 1] 을 시도해보고 가장 좋은 점수가 나온 alpha 값을 과제 제출폼에 제출하세요.**


 -> 값 확인!! 다시!!

In [13]:
from sklearn.linear_model import RidgeCV

alphas = [0, 0.001, 0.01, 0.1, 1]

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5)
ridge.fit(X_train_selected, y_train)
y_pred = ridge.predict(X_test_selected)

print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

alpha:  0.001
best score:  0.5510507608242425


### 5) 테스트셋에 대한 MAE 값을 계산하고 모델이 잘 만들어졌는지 확인해 보세요.
- **테스트셋에 대한 MAE 값을 과제 제출폼에 제출하세요.** -> 값 확인!! 다시!!



In [14]:
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae, r2

(262857.3198109039, 0.5691390939752934)

## 🔥 도전과제

### 공지로 안내드리는 Github - Discussion 페이지에 올려보세요:)

### 6) 본인만의 아이디어로 best ridge regression model을 만들어 서로 공유하시고 토론해 보세요. 
    - 어떤 특성공학을 사용했고, 그 이유와 기대효과는 무엇이었나요? 
    - 모델 성능 평가 지표는 무엇으로 선정했고, 선정한 이유는 무엇인가요? 
    - 어떠한 머신러닝 라이브러리를 사용했고, 어떤 튜닝을 진행했나요? 테스트셋의 성능 변화는 어떠한가요? 
    - 과제 수행 중 부딪혔던 어려움이나 모르는 점이 있었나요? 있었다면 어떻게 해결해 볼 수 있을까요? 